In [162]:
import argparse
import pickle
from collections import defaultdict
from collections import Counter
#from ipdb import set_trace
#from negative_samples import negative_sampler
import numpy as np
import os
#from sma_toolkit import embeddings as emb_utils 
#import streaming_pickle as stPickle
import time
import random
import tensorflow as tf
import parameters

np.set_printoptions(threshold=np.nan)
MIN_DOC_LEN=4

Create Negative Samples that are user specific and then using negative samples and the previous user word ids create user specific train data. create_user_train function is similar to the batch process of skigram model of word2vec.

In [163]:
embed = "word2vec_nce.model"
train_file = "user_text_sample.txt"
output_pkl = "train_embeddings.pkl"
vocabulary_size = 100000
min_word_freq = 5
seed = 42
neg_samples = 10
output = "user_train_data.pkl"

In [164]:
def load_embeddings(filename, wordDict, embedding_size):
    dictionary, steps, word_embeds = pickle.load(open(filename, 'rb'))        
    
    print("dictionary length",len(dictionary))
    print("dictionary keys",len(dictionary.keys()))
    
    key = list(wordDict.keys())[0]
    print('word dict key: ', key)
    print('word dict val: ', wordDict[key])
    
    print("dictionary keys",list(dictionary.keys())[0])
    print("dictionary vals",list(dictionary.values())[0])
    
    print("word embeds: ", len(word_embeds))
    print("word embeds: ", len(word_embeds[0]))
    
    embedding_array = np.zeros((len(wordDict), embedding_size))
    knownWords = list(wordDict.keys())
    unknownWords = []
    
    #print(knownWords)
    foundEmbed = 0
    for i in range(len(embedding_array)):
        index = -1
        
        w = knownWords[i]
        if w in dictionary:
            index = dictionary[w]
        elif w.lower() in dictionary:
            index = dictionary[w.lower()]
        if index >= 0:
            foundEmbed += 1
            embedding_array[i] = word_embeds[index]
        else:
            unknownWords.append(w)
#             embedding_array[i] = np.random.rand(embedding_size) * 0.02 - 0.01
    #print "Found embeddings: ", foundEmbed, "/", len(knownWords)
    #print "embedding_array shape",embedding_array.shape

    print('embed array len: ', len(embedding_array))
    print('embed array len: ', len(embedding_array[0]))
    return embedding_array, dictionary, unknownWords


In [165]:
def get_neg_samples(user_dict, wc):
    neg_dict = defaultdict(list)
    #sample = np.random.choice(vocabulary_size, num_sampled, p=unigram_prob, replace=False)
   
    for user, message in user_dict.items():
        user_wrd = set(message)
        word_corpus = set(wc.keys())
        diff = word_corpus - user_wrd        
        neg_dict[user] = random.sample(diff, parameters.sent_idx)
    return neg_dict

In [166]:
# taken from negative_samples.py of samiroid
def multinomial_samples(unigram_distribution, exclude=[], n_samples=1):
    samples = []        
    while len(samples) != n_samples:            
        wrd_idx = np.argmax(np.random.multinomial(1, unigram_distribution))
        # from ipdb import set_trace; set_trace()
        if wrd_idx not in exclude: 
            samples.append(wrd_idx)                        
    return samples

In [183]:
def create_embeds():
    t0 = time.time()
    word_counter = Counter()
    n_docs = 0
    embedding_size = 128
    user_dict = defaultdict(list)
    c = 0
    with open(train_file,"r") as fid:
        for line in fid:
#             print("len", len(line))
            if len(line) > 1:
                message = line.split()
                user_dict[message[0]].extend(message[1:])
                word_counter.update(message[1:])
                n_docs += 1
    
    #keep only words that occur at least min_word_freq times
    wc = {w:c for w,c in word_counter.items() if c > min_word_freq} 
    #finding unigram probability
    unigram_cnt = [c for w, c in wc.items()]
    total = sum(unigram_cnt)
    unigram_prob = [c*1.0/total for c in unigram_cnt]
        
    tw = sorted(wc.items(), key=lambda x:x[1],reverse=True)
    
    top_words = {w[0]:i for i,w in enumerate(tw[:vocabulary_size])}
    print(type(top_words))    

    embed_matrix, dictionary, unknownWords = load_embeddings(embed, top_words, embedding_size)
    
    print('unknown words len: ', unknownWords)
    print('top words len before: ', top_words)

    for w in unknownWords:
        del top_words[w]
    
    print('top words len after: ', len(top_words))

    wrd2idx = {w:i for i,w in enumerate(top_words.keys())}
#     get_OOEVs(embed_matrix, wrd2idx)
    
    #full_E, full_wrd2idx = emb_utils.read_embeddings(args.emb,top_words)        
#     embeddings = tf.Variable(embed_matrix, dtype=tf.float32)
#     print(embeddings)
    #generate the embedding matrix
    print("embed_matrix shape",embed_matrix.shape)
    emb_size = embed_matrix.shape[0]
    E = np.zeros((int(emb_size), len(wrd2idx)))
    for wrd,idx in wrd2idx.items(): 
        E[:, idx] = embed_matrix[:,top_words[wrd]]
    print("E shape",E.shape)
    pickle.dump([E,unigram_prob,wrd2idx,word_counter,len(user_dict.keys())], open(output_pkl, 'wb'))
    
    return user_dict, wc, wrd2idx, n_docs, unigram_prob
    

In [187]:
def create_user_train(user_dict, wc, wrd2idx, n_docs, unigram_prob):
#     negative_samples = get_neg_samples(user_dict, wc)
    #print(negative_samples)
#     negative_samples = multinomial_samples(unigram_prob, [], 10)
#     print('negative samples: ', negative_samples)

    prev_user, prev_user_data, prev_ctxscores, prev_neg_samples  = None, [], [], []
    full_train = []
#     f_train = open(output,"wb")
    rng = np.random.RandomState(seed)
    with open(train_file, "r") as fid:
        for j, line in enumerate(fid):
            if len(line) <= 1:
                continue

            message = line.lower().split()
            user_id = message[0]
            content = message[1:]

            negative_samples = multinomial_samples(unigram_prob, [], 10)
            #print('negative samples: ', negative_samples)

            #convert to indices
            msg_idx = [wrd2idx[w] for w in content if w in wrd2idx]
#             neg_idx = [wrd2idx[w] for w in negative_samples[user_id] if w in wrd2idx]
            
            if prev_user == None:  # first user
                prev_user = user_id
            elif user_id != prev_user or j == n_docs - 1: # this user_id is seen for the first time

#                 print('user data len: ', len(prev_user_data), 'prev neg len: ', len(prev_neg_samples))
                assert len(prev_user_data) == len(prev_neg_samples)

#                 if len(prev_user_data) > 0:
                # get numbers in range [0, len(prev_user_data))
                shuf_idx = np.arange(len(prev_user_data))
                # shuffle numbers
                rng.shuffle(shuf_idx)

                # fill these lists with the same data in a different order
                prev_user_data = [prev_user_data[i] for i in shuf_idx]
                prev_neg_samples = [prev_neg_samples[i] for i in shuf_idx]
                
                #uncomment the if-else if train is len 0
                # 90-10 train-test split
                split = int(len(prev_user_data)*.9)
#                 if len(prev_user_data) > 1:
                train = prev_user_data[:split]
                test  = prev_user_data[split:]
#                 else:
#                 train = prev_user_data
#                 test = prev_user_data

#                 print('train: ', train)
                    
                neg_samples = prev_neg_samples[:split]
                #print("neg_samples",neg_samples)
                # each training instance consists of:
                # [user_name, train docs, test docs, negative samples]
                full_train.append([prev_user, train, test, neg_samples])
                prev_user_data = []
                prev_neg_samples = []

            prev_user = user_id
#             if len(msg_idx) > 0:
            prev_user_data.append(msg_idx)
#             if len(neg_idx) > 0:
            prev_neg_samples.append(negative_samples)
#             if j == 3:
#                 break

        print('prev user: ', prev_user)
        print('prev user data: ', len(prev_user_data))
        print('prev neg samples: ', len(prev_neg_samples))
    pickle.dump(full_train, open(output, 'wb'))
   

In [188]:
user_dict, wc, wrd2idx, n_docs, unigram_prob = create_embeds()
create_user_train(user_dict, wc, wrd2idx, n_docs, unigram_prob)

<class 'dict'>
dictionary length 100000
dictionary keys 100000
word dict key:  about
word dict val:  30
dictionary keys obedient
dictionary vals 41768
word embeds:  100000
word embeds:  128
embed array len:  78
embed array len:  128
unknown words len:  ["it's", "you're", "don't", "doesn't"]
top words len before:  {'about': 30, 'but': 25, 'same': 66, 'more': 43, 'was': 44, 'than': 53, 'game': 75, 'at': 42, 'of': 5, 'how': 67, 'all': 21, 'who': 73, 'be': 16, 'because': 28, 'they': 15, 'The': 38, 'the': 0, 'will': 29, 'people': 47, 'But': 33, 'as': 19, 'now': 46, 'my': 52, 'is': 4, 'do': 45, 'has': 70, 'Yeah': 71, "it's": 48, 'for': 10, 'know': 34, 'he': 68, 'some': 55, 'them': 72, 'like': 24, 'on': 11, 'only': 56, 'in': 12, 'are': 17, 'so': 20, 'there': 49, 'You': 57, 'want': 58, 'can': 59, "you're": 61, 'up': 39, 'a': 1, 'no': 35, 'what': 31, 'not': 14, 'have': 13, 'his': 60, 'your': 37, 'other': 40, 'just': 23, 'when': 41, 'even': 69, 'it': 9, 'its': 62, 'that': 7, 'this': 18, 'their':